# Start

In [14]:
import os
from pathlib import Path

PROJECT_DIR  = '/content'
os.chdir(Path(PROJECT_DIR))
print("Current Working Directory:", Path.cwd())

Current Working Directory: /content


In [15]:
!if [ -d "pdm" ]; then rm -rf pdm; fi

In [16]:
!git clone https://github.com/soroushdty/pdm.git
REPO_DIR = Path.cwd() / "pdm"
print(REPO_DIR)
os.chdir(REPO_DIR)
print("Current Working Directory:", Path.cwd())

Cloning into 'pdm'...
remote: Enumerating objects: 228, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (66/66), done.
Receiving objects: 100% (228/228), 100.27 KiB | 580.00 KiB/s, done.
remote: Total 228 (delta 42), reused 2 (delta 2), pack-reused 160 (from 1)
Resolving deltas: 100% (123/123), done.
/content/pdm
Current Working Directory: /content/pdm


In [17]:
from funcs.requirements_utils import install_missing
installed = install_missing(Path.cwd() / "requirements.txt", quiet=False)
print("Installed:", installed)

Installed: []


In [18]:
# Load config
from funcs.config_loader import load_config
CONFIG_PATH = Path.cwd() / "config.json"
cfg = load_config(CONFIG_PATH)
cfg

{'DIR_INPUT': 'input',
 'DIR_OUTPUT': 'output',
 'patient_col': 'Patient',
 'physician_col': 'Physician',
 'item_col': 'Item',
 'classes': ['Behavioral health',
  'Diagnoses',
  'Disabilities',
  'Infectious diseases',
  'Genetics',
  'Medications',
  'Sexual and reproductive health',
  'Social determinants of health',
  'Violence',
  'Other'],
 'other_class': ['Disabilities',
  'Infectious diseases',
  'Genetics',
  'Sexual and reproductive health',
  'Violence',
  'Social determinants of health'],
 'llms': ['sentence-transformers/all-MiniLM-L6-v2',
  'sentence-transformers/all-mpnet-base-v2',
  'sentence-transformers/multi-qa-MiniLM-L6-cos-v1',
  'pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb',
  'sentence-transformers/biomed-mpnet-base',
  'cambridgeltl/SapBERT-from-PubMedBERT-fulltext',
  'bert-base-uncased',
  'roberta-base',
  'distilbert-base-uncased',
  'nreimers/MiniLM-L6-H384-uncased',
  'emilyalsentzer/Bio_ClinicalBERT',
  'monologg/biobert_v1.1_pubmed',
  'bionlp/

# run

In [19]:
# Create run directory + logger, then run pipeline
from funcs.paths import make_run_dir
from funcs.logger_setup import setup_logger
from funcs.preprocessing import run

run_dir = make_run_dir(Path.cwd() / cfg["DIR_OUTPUT"])
logger = setup_logger(run_dir / "log.txt")
p = run(cfg)

[23:10:44 02-01-26] INFO: Logging initialized at: /content/pdm/output/23-10_FEB-01/log.txt


FileNotFoundError: Dataset directory not found at: /content/pdm/DIR_INPUT

In [ ]:

train = p['paths']['train_csv']
test = p['paths']['test_csv']
train_map = p['paths']['train_map']
test_map = p['paths']['test_map']

FileNotFoundError: Dataset file not found at: input/dataset.xlsx